In [23]:
import pretty_midi
import random
import numpy as np


In [18]:
circle_of_fifths_chords_major = {
    "C":  [60, 64, 67],   # C major
    "G":  [67, 71, 74],   # G major
    "D":  [62, 66, 69],   # D major
    "A":  [69, 73, 76],   # A major
    "E":  [64, 68, 71],   # E major
    "B":  [71, 75, 78],   # B major
    "F#": [66, 70, 73],   # F# major
    "Db": [61, 65, 68],   # Db major
    "Ab": [68, 72, 75],   # Ab major
    "Eb": [63, 67, 70],   # Eb major
    "Bb": [70, 74, 77],   # Bb major
    "F":  [65, 69, 72]    # F major
}

In [19]:
circle_of_fifths_chords_minor = {    
    "Am":  [69, 72, 76],
    "Em":  [64, 67, 71],
    "Bm":  [71, 74, 78],
    "F#m": [66, 69, 73],
    "C#m": [61, 64, 68],
    "G#m": [68, 71, 75],
    "D#m": [63, 66, 70],
    "Bbm": [70, 73, 77],
    "Fm":  [65, 68, 72],
    "Cm":  [60, 63, 67],
    "Gm":  [67, 70, 74],
    "Dm":  [62, 65, 69]
}

In [20]:
circle_of_fifth_total ={
    "C":  [60, 64, 67],   # C major
    "G":  [67, 71, 74],   # G major
    "D":  [62, 66, 69],   # D major
    "A":  [69, 73, 76],   # A major
    "E":  [64, 68, 71],   # E major
    "B":  [71, 75, 78],   # B major
    "F#": [66, 70, 73],   # F# major
    "Db": [61, 65, 68],   # Db major
    "Ab": [68, 72, 75],   # Ab major
    "Eb": [63, 67, 70],   # Eb major
    "Bb": [70, 74, 77],   # Bb major
    "F":  [65, 69, 72],    # F major
    "Am":  [69, 72, 76],
    "Em":  [64, 67, 71],
    "Bm":  [71, 74, 78],
    "F#m": [66, 69, 73],
    "C#m": [61, 64, 68],
    "G#m": [68, 71, 75],
    "D#m": [63, 66, 70],
    "Bbm": [70, 73, 77],
    "Fm":  [65, 68, 72],
    "Cm":  [60, 63, 67],
    "Gm":  [67, 70, 74],
    "Dm":  [62, 65, 69]

}

In [176]:
def generate_inversion(root_chord):
    root, third, fifth = root_chord
    return {
        "root": [root, third, fifth],
        "1st": [third, fifth, root +12],
        "2nd": [fifth, root +12, third +12]

    }

circle_rules = {
    "C":   ["G", "F", "Am", "Cm"],
    "G":   ["D", "C", "Em", "Gm"],
    "D":   ["A", "G", "Bm", "Dm"],
    "A":   ["E", "D", "F#m", "Am"],
    "E":   ["B", "A", "C#m", "Em"],
    "B":   ["F#", "E", "G#m", "Bm"],
    "F#":  ["Db", "B", "D#m", "F#m"],
    "Db":  ["Ab", "F#", "Bbm", "C#m"],
    "Ab":  ["Eb", "Db", "Fm", "G#m"],
    "Eb":  ["Bb", "Ab", "Cm", "D#m"],
    "Bb":  ["F", "Eb", "Gm", "Bbm"],
    "F":   ["C", "Bb", "Dm", "Fm"],

    "Am":  ["Em", "Dm", "C", "A"],
    "Em":  ["Bm", "Am", "G", "E"],
    "Bm":  ["F#m", "Em", "D", "B"],
    "F#m": ["C#m", "Bm", "A", "F#"],
    "C#m": ["G#m", "F#m", "E", "C#"],
    "G#m": ["D#m", "C#m", "B", "G#"],
    "D#m": ["Bbm", "G#m", "F#", "D#"],
    "Bbm": ["Fm", "D#m", "Db", "Bb"],
    "Fm":  ["Cm", "Bbm", "Ab", "F"],
    "Cm":  ["Gm", "Fm", "Eb", "C"],
    "Gm":  ["Dm", "Cm", "Bb", "G"],
    "Dm":  ["Am", "Gm", "F", "D"]
}

circle_of_fifth_inversions = {
    chord: generate_inversion(notes)
    for chord, notes in circle_of_fifth_total.items()
}

In [177]:
def generate_circle_of_fifth(start_chord=None, num_chords = None, rules = circle_rules):
    if start_chord is None:
        start_chord = start_chord=random.choice(list(circle_rules.keys()))
    if num_chords is None:
        num_chords = np.random.randint(4,8)
    
    progression = [start_chord]
    current = start_chord
    for _ in range(num_chords - 1):
        options = rules.get(current, [])
        if not options:
            break
        next_chord = random.choice(options)
        progression.append(next_chord)
        current = next_chord
    return progression


In [178]:
def write_progression_to_midi(
    chord_progression,
    chord_dict,
    filename="generated_progression_with_bass.mid",
    duration=2.0,
    velocity=None,
    inversion="random"
):
    if velocity is None:
        velocity = int(np.random.randint(60, 85))

    pm = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(program=0)  # Acoustic Grand Piano
    current_time = 0.0
    total_notes_written = 0

    for chord in chord_progression:
        if chord not in chord_dict:
            print(f"⚠️ Skipping unknown chord: {chord}")
            continue

        inversions = chord_dict[chord]

        # Select inversion
        if inversion == "random":
            notes = random.choice(list(inversions.values()))
        else:
            notes = inversions.get(inversion, inversions["root"])

        # Transpose all notes down by 12 semitones
        lowered_notes = [p - 12 for p in notes if 21 <= (p - 12) <= 108]

        # Add root note 2 octaves down, if it's in range
        original_root = notes[0]
        bass_note = original_root - 24
        if 21 <= bass_note <= 108:
            final_notes = [bass_note] + lowered_notes
        else:
            final_notes = lowered_notes

        print(f"🎼 {chord} → {final_notes}")

        for i, pitch in enumerate(final_notes):
            is_bass = (i == 0 and pitch == bass_note)
            note_velocity = random.randint(40, 60) if is_bass else random.randint(70, 100)

            note = pretty_midi.Note(
                velocity=note_velocity,
                pitch=pitch,
                start=current_time,
                end=current_time + duration
            )
            piano.notes.append(note)
            total_notes_written += 1

        current_time += duration

    pm.instruments.append(piano)
    pm.write(filename)
    print(f"✅ MIDI file saved as: {filename}")
    print(f"🎹 Total notes written: {total_notes_written}")

In [181]:
progression = generate_circle_of_fifth()
print(progression)

['B', 'F#', 'Db', 'F#', 'F#m']


In [182]:
write_progression_to_midi(
    progression,
    circle_of_fifth_inversions,  # <-- correct position for chord_dict
    filename="wolfie_circle_inversions.mid",
    inversion="random"
)

🎼 B → [47, 59, 63, 66]
🎼 F# → [42, 54, 58, 61]
🎼 Db → [44, 56, 61, 65]
🎼 F# → [49, 61, 66, 70]
🎼 F#m → [42, 54, 57, 61]
✅ MIDI file saved as: wolfie_circle_inversions.mid
🎹 Total notes written: 20
